In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import pandas as pd
import time
import os

In [2]:
service = Service(executable_path='/Applications/Chdriver/chromedriver')
driver = webdriver.Chrome(service=service)

url = 'https://www.cricbuzz.com/cricket-series/2810/indian-premier-league-2019/matches'
driver.get(url)

match_urls = []
matches = driver.find_elements(By.CLASS_NAME, 'cb-series-matches')
for match in matches:
    link_element = match.find_element(By.CLASS_NAME, 'text-hvr-underline').get_attribute('href')
    link = link_element.replace("cricket-scores", "live-cricket-scorecard")
    match_urls.append(link)

batter_data = {
    'Match Date': [],
    'Venue': [],
    'Link': [],
    'Runs': [],
    'Balls': [],
    'Fours': [],
    'Sixes': [],
    'Strike Rate': [],
    'Team': [],
    'Season': []
}

bowler_data = {
    'Match Date': [],
    'Venue': [],
    'Link': [],
    'Overs': [],
    'Maiden': [],
    'Runs': [],
    'Wicket': [],
    'No Ball': [],
    'Wide': [],
    'Economy': [],
    'Team': [],
    'Season': []
}

for url in match_urls:
    driver.get(url)
    time.sleep(1)

    try:
        venue = driver.find_element(By.XPATH, '//*[@id="page-wrapper"]/div[4]/div[1]/div[2]/a[2]/span/span[2]/span')
        
        season_element = driver.find_element(By.XPATH, '//*[@id="page-wrapper"]/div[4]/div[1]/div[2]/a[1]/span')
        season_words = season_element.text.split()[3:]
        season = ' '.join(season_words)
        
        date = driver.find_element(By.CSS_SELECTOR, 'span.schedule-date')
        date_object = pd.to_datetime(date.text, format='%A, %B %d, %Y')
        
        batting1 = driver.find_element(By.XPATH, '//*[@id="innings_1"]/div[1]')
        batting2 = driver.find_element(By.XPATH, '//*[@id="innings_2"]/div[1]')
        
        xpath1 = '//*[@id="innings_1"]/div[1]/div[1]/span[1]'
        xpath2 = '//*[@id="innings_2"]/div[1]/div[1]/span[1]'
        for section, team_xpath in zip([batting1, batting2], [xpath1, xpath2]):
            team_element = section.find_element(By.XPATH, team_xpath)
            team_words = team_element.text.split(' ')[:-1]
            initials = [word[0] for word in team_words]
            team_name = ''.join(initials)
            
            batters = section.find_elements(By.CLASS_NAME, 'cb-scrd-itms')
            for batter in batters[:-3]:
                batter_data['Venue'].append(venue.text)                
                batter_data['Season'].append(season)
                batter_data['Match Date'].append(date_object.strftime('%Y-%m-%d'))

                name_link = batter.find_element(By.CLASS_NAME, 'cb-text-link').get_attribute('href')
                batter_data['Link'].append(name_link)
                                
                scores = batter.find_elements(By.CLASS_NAME, 'text-right')
                batter_data['Runs'].append(scores[0].text)
                batter_data['Balls'].append(scores[1].text)
                batter_data['Fours'].append(scores[2].text)
                batter_data['Sixes'].append(scores[3].text)
                batter_data['Strike Rate'].append(scores[4].text)
                batter_data['Team'].append(team_name)
            
        bowling1 = driver.find_element(By.XPATH, '//*[@id="innings_1"]/div[4]')
        bowling2 = driver.find_element(By.XPATH, '//*[@id="innings_2"]/div[4]')
        xpath1 = '//*[@id="innings_1"]/div[1]/div[1]/span[1]'
        xpath2 = '//*[@id="innings_2"]/div[1]/div[1]/span[1]'
        for section, team_xpath in zip([bowling1, bowling2], [xpath2, xpath1]):
            team_element = section.find_element(By.XPATH, team_xpath)
            team_words = team_element.text.split(' ')[:-1]
            initials = [word[0] for word in team_words]
            team_name = ''.join(initials)
            
            bowlers = section.find_elements(By.CLASS_NAME, 'cb-scrd-itms')
            for bowler in bowlers:
                name_link = bowler.find_element(By.CLASS_NAME, 'cb-text-link').get_attribute('href')   
                bowler_data['Link'].append(name_link)

                bowler_data['Match Date'].append(date_object.strftime('%Y-%m-%d'))               
                bowler_data['Venue'].append(venue.text)
                bowler_data['Season'].append(season)

                scores = bowler.find_elements(By.CLASS_NAME, 'text-right')
                bowler_data['Overs'].append(scores[0].text)
                bowler_data['Maiden'].append(scores[1].text)
                bowler_data['Runs'].append(scores[2].text)
                bowler_data['Wicket'].append(scores[3].text)
                bowler_data['No Ball'].append(scores[4].text)
                bowler_data['Wide'].append(scores[5].text)
                bowler_data['Economy'].append(scores[6].text)
                bowler_data['Team'].append(team_name)
                
    except NoSuchElementException as e:
        pass

batter_df = pd.DataFrame(batter_data)
bowler_df = pd.DataFrame(bowler_data)


driver.quit()

In [3]:
batter_link = pd.DataFrame({'Link': batter_df['Link'].unique()})
bowler_link = pd.DataFrame({'Link': bowler_df['Link'].unique()})
player_link = pd.concat([batter_link, bowler_link], axis=0).reset_index(drop=True)['Link'].unique()

In [4]:
service = Service(executable_path='/Applications/Chdriver/chromedriver')
driver = webdriver.Chrome(service=service)

player_data = {
    'Link': [],
    'Name': [],
    'Country': [],
    'Born': []
}
for link in player_link:
    driver.get(link)
    time.sleep(0.1)
    
    player_data['Link'].append(link)
    
    name = driver.find_element(By.XPATH, '//*[@id="playerProfile"]/div[1]/div[2]/h1').text
    player_data['Name'].append(name)

    country = driver.find_element(By.XPATH, '//*[@id="playerProfile"]/div[1]/div[2]/h3').text
    player_data['Country'].append(country)

    date_string = driver.find_element(By.XPATH, '//*[@id="playerProfile"]/div[2]/div[1]/div/div[3]').text
    date_part = date_string.split('(')[0].strip()
    parsed_date = datetime.strptime(date_part, '%b %d, %Y')
    formatted_date = parsed_date.strftime('%Y-%m-%d')
    player_data['Born'].append(formatted_date)
    
player_df = pd.DataFrame(player_data)    
driver.quit()

In [4]:
player_df

NameError: name 'player_df' is not defined

In [6]:
df1 = pd.merge(batter_df, player_df, on='Link', how='left')
df2 = pd.merge(bowler_df, player_df, on='Link', how='left')
df1 = df1[['Match Date', 'Name', 'Runs', 'Balls','Fours', 'Sixes', 'Strike Rate', 'Country', 'Born', 'Team', 'Venue', 'Season']]
df2 = df2[['Match Date', 'Name', 'Overs', 'Maiden','Runs', 'Wicket', 'No Ball', 'Wide', 'Economy', 'Country', 'Born', 'Team', 'Venue', 'Season']]

In [7]:
batter_file = 'ipl-2019-batter.csv'
bowler_file = 'ipl-2019-bowler.csv'

df1.to_csv(batter_file, index=False, header=not os.path.isfile(batter_file))
df2.to_csv(bowler_file, index=False, header=not os.path.isfile(bowler_file))


In [9]:
df = pd.read_csv(batter_file)
df

,Match Date,Name,Runs,Balls,Fours,Sixes,Strike Rate,Country,Born,Team,Venue,Season
0,2021-04-09,Rohit Sharma,19,15,1,1,126.67,India,1987-04-30,MI,Chennai,2021
1,2021-04-09,Chris Lynn,49,35,4,3,140.00,Australia,1990-04-10,MI,Chennai,2021
2,2021-04-09,Suryakumar Yadav,31,23,4,1,134.78,India,1990-09-14,MI,Chennai,2021
3,2021-04-09,Ishan Kishan,28,19,2,1,147.37,India,1998-07-18,MI,Chennai,2021
4,2021-04-09,Hardik Pandya,13,10,2,0,130.00,India,1993-10-11,MI,Chennai,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
911,2021-10-15,Dinesh Karthik,9,7,0,1,128.57,India,1985-06-01,KKR,Dubai,2021
912,2021-10-15,Shakib Al Hasan,0,1,0,0,0.00,Bangladesh,1987-03-24,KKR,Dubai,2021
913,2021-10-15,Rahul Tripathi,2,3,0,0,66.67,India,1991-03-02,KKR,Dubai,2021
914,2021-10-15,Lockie Ferguson,18,11,1,1,163.64,New Zealand,1991-06-13,KKR,Dubai,2021
